In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd

import joblib

In [3]:
test_data = pd.read_csv('drive/My Drive/Machine_Learning/Final/test.csv', index_col = 'id')
logistic_regression_pipeline = joblib.load('drive/My Drive/Machine_Learning/Final/LR_model')

In [4]:
#怎麼選的部分
#https://www.kaggle.com/code/alvinleenh/tpsoct22-6-basic-feature-selection-techniques?scriptVersionId=107676046
#https://www.kaggle.com/code/desalegngeb/tps08-logisticregression-qlattice/notebook
def create_features(df):
    # https://www.kaggle.com/code/takanashihumbert/tps-aug22-lb-0-59013
    # missing3 and missing5
    df['m3_missing'] = df['measurement_3'].isnull().astype(np.int8)
    df['m5_missing'] = df['measurement_5'].isnull().astype(np.int8)
    
    # area
    df['area'] = df['attribute_2'] * df['attribute_3']
    # https://www.kaggle.com/code/desalegngeb/tps08-logisticregression-qlattice/notebook
    df['missing(3*5)'] = df['m3_missing'] * df['m5_missing']
    #df['measurement(3*5)'] = df['measurement_3'] * df['measurement_5']

    # 
    df['measurement_0 / loading'] = df['measurement_0'] / df['loading']
    df['measurement_1 / loading'] = df['measurement_1'] / df['loading']
    df['measurement_2 / loading'] = df['measurement_2'] / df['loading']
    # measure17重要，其他取平均就好
    df['measurement_3_to_16_mean / loading'] = df[[f"measurement_{x:d}" for x in range(3,17)]].mean(axis='columns') / df['loading']
    df['measurement_17 / loading'] = df['measurement_17'] / df['loading']

In [5]:
X_columns = [ # 0.59184
    'loading',
    'attribute_0',
    'attribute_1',

    'm3_missing',
    'm5_missing',
    'area',
    'missing(3*5)',
     #'measurement(3*5)',

    'measurement_0 / loading',
    'measurement_1 / loading',
    'measurement_2 / loading',
    'measurement_3_to_16_mean / loading',
    'measurement_17 / loading',
]

In [6]:
create_features(test_data)

In [7]:
test_data_predict_prob = logistic_regression_pipeline.predict_proba(test_data[X_columns])[:, 1]

In [9]:
output = pd.DataFrame({'id': test_data.index, 'failure': test_data_predict_prob})
output.to_csv('109550137_submission.csv', index=False)